In [1]:
data = [1.0, 2.0, 1.5, 4.5, 1.25, 1.75] ## A good fit to this would be a background of 1.5 and a spike of 3.0 in the fourth position

In [2]:
background = 1.0

In [3]:
spike_position = 3.75

In [4]:
spike_flux = 4.0

In [5]:
def make_model(background, spike_position, spike_flux):
    def f(i):
        distance = abs(i - spike_position)
        height = 0.0
        if distance < 1.0:
            height = (1 - distance) * spike_flux
        return background + height
    return f


In [6]:
model = make_model(background, spike_position, spike_flux)

In [7]:
def variance(model, data):
    sum = 0.0
    for i in range(len(data)):
        difference = model(i) - data[i]
        sum += difference**2
    return sum
        

In [8]:
variance(model, data)

15.625

In [9]:
from statistics import median

In [10]:
background_guess = median(data)

In [11]:
maximum = max(data)

In [12]:
spike_flux_guess = maximum - background_guess

In [13]:
spike_position_guess = data.index(maximum)